In [1]:
from google.colab import files
files.upload()   # select kaggle.json here

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"michealjerinas","key":"f01fc278e1da9cef18b50fedb81c4c93"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d tongpython/cat-and-dog
!unzip -q cat-and-dog.zip -d /content/cat-dog


Dataset URL: https://www.kaggle.com/datasets/tongpython/cat-and-dog
License(s): CC0-1.0
 74% 162M/218M [00:00<00:00, 635MB/s] 
100% 218M/218M [00:00<00:00, 524MB/s]


In [4]:
train_dir = "/content/cat-dog/training_set/training_set"
categories = ["cats", "dogs"]


In [6]:
import os
import cv2
import numpy as np

data = []
labels = []
img_size = 100 # Define img_size

for category in categories:
    folder = os.path.join(train_dir, category)
    class_num = categories.index(category)  # 0=cats, 1=dogs
    for img in os.listdir(folder)[:300]:   # limit to 300 per class
        try:
            img_path = os.path.join(folder, img)
            img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # grayscale
            img_array = cv2.resize(img_array, (img_size, img_size))
            data.append(img_array.flatten())  # flatten to feature vector
            labels.append(class_num)
        except:
            continue

X = np.array(data)
y = np.array(labels)

print("Dataset shape:", X.shape, y.shape)

Dataset shape: (600, 10000) (600,)


In [7]:
img_size = 32   # instead of 64


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, accuracy_score

# Split into 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train fast linear SVM (SGD approximation)
svm = SGDClassifier(loss="hinge", max_iter=1000, tol=1e-3, random_state=42)
svm.fit(X_train, y_train)

# Predictions
y_pred = svm.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=categories))


Accuracy: 0.49166666666666664
              precision    recall  f1-score   support

        cats       0.51      0.58      0.54        62
        dogs       0.47      0.40      0.43        58

    accuracy                           0.49       120
   macro avg       0.49      0.49      0.49       120
weighted avg       0.49      0.49      0.49       120



In [10]:
def predict_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (100, 100)) # Use 100x100 as used for training
    img = img.flatten().reshape(1, -1)
    pred = svm.predict(img)[0]
    return categories[pred]

# Example: pick one image from the test set
test_path = "/content/cat-dog/test_set/test_set/cats/cat.4001.jpg"
print("Prediction:", predict_image(test_path))

Prediction: dogs


In [13]:
from google.colab import files
uploaded = files.upload()   # choose your image from PC


Saving dog.jpg to dog.jpg


In [14]:
test_path = list(uploaded.keys())[0]   # get the uploaded file name
print("Prediction:", predict_image(test_path))


Prediction: cats
